In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

// TODO(Johann) could these be moved to json configuration files?
val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val IMPORT_CONFIGURATION_FILE = "../../data/provision/import-configuration.json"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-12.0"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=5a724030-2d90-41b1-b776-520920d7fdd5, username=Johann-12.0)

In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=20769538-ec34-470c-8768-f2220ab92287, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=f484d20f-7f70-4e36-accb-dc9c56aef9fd, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=25516e82-5025-4233-919f-8c4ea57e7b80, name=ING RON, unit=Currency(value=RON))
AccountTO(id=c0f90f76-21bc-42e7-9e5d-b1e8548f4955, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=eb6049a9-7132-4970-9c55-588cb2dc4fd4, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=978b491b-a7d8-477d-8e6f-f07e891dd26e, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=f4cf1850-f363-4b81-8bad-b925dff7798a, name=Other, unit=Currency(value=RON))
AccountTO(id=eacc15d3-41e5-430e-9287-05a619467684, name=BT RON, unit=Currency(value=RON))
AccountTO(id=5655c050-8457-411d-8c32-e5d224cabc7e, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=8e952d91-0efb-4b52-9ae4-798a00b9bc70, name=Expenses)
FundTO(id=cd34eee0-5721-4773-b04a-9507ba1dc874, name=Work Income)
FundTO(id=4606428d-0672-409b-bd85-35872527209a, name=Savings)
FundTO(id=812ae35f-b7c3-412e-a38b-6e62b24e0b62, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawJson: String = File(IMPORT_CONFIGURATION_FILE).readText()
val jsonFormat = Json { ignoreUnknownKeys = true }
val importConfiguration = jsonFormat.decodeFromString<ImportConfigurationTO>(dataConfigurationRawJson)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles = File("../../data/wallet/2019")
    .listFiles()?.toList() ?: error("no files found")
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=f86e3f91-e035-4bf3-981b-e87a7ecffa26, status=COMPLETED, reason=null)